# _Gradient descent_

We zagen tot nu toe dat we parameters van een lineair regressiemodel kunnen schatten via:

- **_Grid search_**: We zoeken exhaustief een deel van de parameterruimte af naar optimale waarden voor de SSE _loss_. Zeker bij complexere modellen ($\pmb{b} = \begin{bmatrix}b_1, b_2, \ldots, b_n\end{bmatrix}$) is dit praktisch onhaalbaar.
- **_Monte Carlo sampling_**: Hier leveren we ons over aan het _willekeurig aftasten_ van de $SSE$ functie. Dit levert zelfs bij simpele modellen geen stabiele resultaten op.
- **_Ordinary least squares_** (_OLS_): Deze puur analytische oplossing kan in één keer een optimale oplossing opleveren (in de _least squares_ zin), maar kan onstabiele resultaten opleveren wanneer er sprake is van multicollineariteit.

In deze sectie bespreken we een populair alternatief: **_gradient descent_**. Zeker in de context van complexere ML modellen (bv. neurale netwerken) biedt kan deze oplossing een stabieler alternatief. Zoals de naam zelf suggereert, maakt de _gradient descent_ oplossing, net zoals OLS gebruik van de gradient.

:::{important} Gradient
Herinner je dat de **gradient** in deze context de **tensor is met alle partiële afgeleiden van de _loss_ functie bij een gegeven set van parameterwaarden** $\nabla J(\pmb{b}^k)$. Ieder individueel element staat voor de richting van de raaklijn met de _loss_ (hyper) _surface_ voor een bepaalde parameter ($b^k_i$), in de veronderstelling dat alle andere parameters constant blijven (de partiële afgeleide: $\frac{\partial J(\pmb{b}^k)}{\partial b^k_i}$).
  
De grootte van de individuele elementen geeft aan hoe sterk de _loss_ functie toeneemt (bij een positieve waarde) of afneemt (bij een negatieve waarde) als de overeenkomstige parameterwaarde ($b^k_i$) met een bepaalde stapgrootte wordt verschoven. Let wel dat het de toe-/afname betreft volgens de lokale lineaire benadering (vandaar geen $=$ maar $\sim$ hieronder): 

$$
J(b^{k+1}_i) \sim J(b^k_i) + \frac{\partial J(\pmb{b}^k)}{\partial b^k_i} \Delta b^{k+1}_i
$$

met

$$
\Delta b^{k+1}_i = b^{k+1}_i - b^k_i
$$
:::

Bij _gradient descent_ is het basisidee dat we, zoals bij de Monte Carlo benadering, iteratief parameterwaarden aanpassen in de richting waarin de _loss_ daalt. Alleen laten we dit nu niet afhangen van een random zoektocht, maar gebruiken we de gradient om ons te leiden. We beginnen met een initiële schatting $\pmb{b}_0$. Daarna gaan we die schatting iteratief updaten met de volgende regel:

$$
\pmb{b}^{k+1} = \pmb{b}^k - \lambda \nabla J(\pmb{b}^k) 
$$
waarbij de hyper parameter $\lambda$ de **_learning rate_** is met $\lambda > 0$
  
Met andere woorden:
- **bij iedere update, verschuiven we _alle_ parameterwaarden in de richting van de sterkste daling van de _loss_ functie**
- **de grootte van de update wordt bepaald als een fractie van de gradient (of de individuele partiële afgeleiden), gecontroleerd door de _learning rate_ $\lambda$**

::: {note}
Om de formule beter te begrijpen kunnen we ons een situatie voorstellen waarbij de _learning rate_ $1$ is en:
1. _de partiële afgeleide $1$ is_: de raaklijn stijgt met een slope $1$ van links naar rechts op de schaal van onze parameter $b_i$
2. _de partiële afgeleide $-1$ is_: de raaklijn daalt met een slope $-1$ van links naar rechts op de schaal van onze parameter $b_i$  

Om onze parameter $b_i$ aan te passen in de richting van een lagere _loss_ waarde moeten we die dan bij (1.) naar links verschuiven en bij (2.) naar rechts. Als we de formule toepassen krijgen we inderdaad bij (1.) een verschuiving van $-1$ en bij (2.) van $+1$.
:::



In [ ]:
# illustration

## Stochastische _gradient descent_